## 1. Import Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print("Libraries imported.")

Libraries imported.


## Collect Data from Wikipedia Page into DataFrame

In [4]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [11]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [12]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


## Drop the borough cells which are not assigned

In [13]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront
3,M6A\n,North York\n,Lawrence Manor / Lawrence Heights
4,M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government


## Group Neighbourhoods in the same borough

In [14]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge\n"
1,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G\n,Scarborough\n,"Woburn, Woburn\n"
4,M1H\n,Scarborough\n,"Cedarbrae, Cedarbrae\n"


## For Neighborhood="Not assigned", make the value the same as Borough

In [15]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge\n"
1,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G\n,Scarborough\n,"Woburn, Woburn\n"
4,M1H\n,Scarborough\n,"Cedarbrae, Cedarbrae\n"


## Print the shape

In [17]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)

## Load the coordinates from the csv file 

In [23]:
# load the coordinates from the csv file on Coursera
coordinates = pd.read_csv("/Users/tutik/Downloads/Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge two tables to get the coordinates

In [28]:
# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge\n",NaN,NaN
1,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug...",NaN,NaN
2,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood...",NaN,NaN
3,M1G\n,Scarborough\n,"Woburn, Woburn\n",NaN,NaN
4,M1H\n,Scarborough\n,"Cedarbrae, Cedarbrae\n",NaN,NaN


In [26]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
